<h2><center>Supreme Court Oral Arguments Outcome Prediction Team</center></h2>
<h2 style="margin-top: -15px; margi-bottom: 10px;"><center>Final Report</center></h2>

### Project and Team Information
**Team members:** Federico Dominguez Molina, Jessup Jong, Chanteria Milner, and Michael Plunkett

**Project summary:** The project uses historic United States Supreme Court cases to train natural language processing models to predict case rulings. \
**Project repository:** [Link](https://github.com/michplunkett/supreme-court-ml-predictions) \
**Project assumptions and things to know:**
1. The number of unique roles within the advocates' file is too numerous to be helpful, so we merged them into 5 categories. While this merger may remove some variability and nuance in the file, we believe it will make it easier to derive meaningful conclusions.
    - The groupings for the roles are as follows: inferred, for respondent, for partitioner, and for amicus curiae
2. The years included within this data set are 2014 to 2019.
3. The datasets included within the previously mentioned year range are ones where the winnings side was either 0 or 1 (no missing, etc.).

**Models run:** Logistic Regression, XG Boost, and Random Forest

In [2]:
### Imports and setup
import jupyter_black

jupyter_black.load()

ModuleNotFoundError: No module named 'jupyter_black'